In [ ]:
# !pip install -q --upgrade pip
# !pip install -q sagemaker smdebug awscli sagemaker-experiments --upgrade

In [1]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [2]:
import time
import boto3
sess = boto3.Session()
sm = sess.client('sagemaker')

In [3]:
def cleanup_experiment(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(sagemaker_boto_client=sm, trial_name=trial_summary.trial_name)
        try:
            for trial_component_summary in trial.list_trial_components():
                tc = TrialComponent.load(
                    sagemaker_boto_client=sm,
                    trial_component_name=trial_component_summary.trial_component_name)
                try:
                    trial.remove_trial_component(tc)
                    print(f'Deleting trial component: {tc.trial_component_name}')
                    tc.delete()
                except:
                    time.sleep(0.4)
                    continue
            print(f'Deleting trial: {trial.trial_name}')
            trial.delete()
        except:
            continue
    print(f'Deleting experiment: {experiment.experiment_name}')
    experiment.delete()

#### List of existing experiments

In [38]:
exp_names = [exp.experiment_name for exp in Experiment.list()]
print('Current experiments:')
exp_names

Current experiments:


[]

In [39]:
for exp in exp_names:
    cleanup_experiment(Experiment.load(exp))

#### Delete unassigned trial components

In [45]:
for tc in TrialComponent.list():
    try:
        sm.delete_trial_component(TrialComponentName=tc.display_name)
        print(f'Deleted: {tc.display_name}')
        time.sleep(0.4)
    except:
        continue